# Multiple Latent

Trying to use more than 1 latent variable

In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
from meteo_imp.gpfa.imputation import *
from meteo_imp.data_preparation import *
from meteo_imp.results import *

import torch
import pandas as pd
import numpy as np
from pathlib import Path
from pyprojroot import here
import matplotlib.pyplot as plt

from meteo_imp.utils import cache_disk

In [ ]:
def reset_seed():
    torch.manual_seed(27);
    np.random.seed(27);

In [ ]:
torch.manual_seed(27);
np.random.seed(27);
cache_path = here() / ".cache"

## Load Data

take the first 200 rows from the Hainich dataset

In [ ]:
hai_path = Path("FLX_DE-Hai_FLUXNET2015_FULLSET_HH_2000-2012_1-4.csv")
hai_raw = pd.read_csv(here("data") / hai_path, na_values=["-9999", "-9999.99"], parse_dates=[0, 1], nrows=200)

In [ ]:
meteo_vars = {
    "TA_F": "TA",
    "SW_IN_F": "SW_IN",
    "LW_IN_F": "LW_IN",
    "VPD_F": "VPD",
    #"PA": "PA"
}

units = {
    'TA': '°C',
    'SW_IN': 'W m-2',
    'LW_IN': 'W m-2',
    'VPD': 'hPa'
}

hai = (hai_raw
       .rename(columns=meteo_vars)
       .set_index("TIMESTAMP_END")
       .loc[:, meteo_vars.values()])
hai

,TA,SW_IN,LW_IN,VPD
TIMESTAMP_END,,,,
2000-01-01 00:30:00,-0.60,0.0,302.475,0.222
2000-01-01 01:00:00,-0.65,0.0,302.475,0.122
2000-01-01 01:30:00,-0.58,0.0,301.677,0.090
2000-01-01 02:00:00,-0.51,0.0,301.677,0.110
2000-01-01 02:30:00,-0.49,0.0,301.677,0.102
...,...,...,...,...
2000-01-05 02:00:00,4.74,0.0,330.202,1.191
2000-01-05 02:30:00,4.75,0.0,330.202,1.057
2000-01-05 03:00:00,4.76,0.0,330.202,0.935


###  Computation

makes here all the slow computations and cache them on disk

In [ ]:
reset_seed()
data_r_gaps = GPFADataTest(hai[:150]).add_random_missing()
data_c_gaps = GPFADataTest(hai[:150]).add_gap(15, meteo_vars.values())

In [ ]:
cache_file_gaps = cache_path / "hai_diff_latents.pickle"
# cache_file_gaps.unlink() # uncomment this line to reset the cache

In [ ]:
@cache_disk(cache_file_gaps)
def compute_multiple_latent():
    hai_r_gaps = [GPFAImputation(
        data_r_gaps.data, latent_dims=i)
                  .fit()
                  .to_result(data_r_gaps.data_compl_tidy, units=units)
                  for i in range(1,4)]
    hai_c_gaps = [GPFAImputationExplorer(
        data_c_gaps.data, latent_dims=i)
                  .fit()
                  .to_result(data_c_gaps.data_compl_tidy, units=units)
                  for i in range(1,4)]
    return hai_r_gaps, hai_c_gaps

In [ ]:
hai_r_gaps, hai_c_gaps = compute_multiple_latent()

## Random Gaps

In [ ]:
hai_r_gaps

[<gpfa_imputation.results.GPFAResult>,
 <gpfa_imputation.results.GPFAResult>,
 <gpfa_imputation.results.GPFAResult>]

In [ ]:
hai_r_gaps[0].display_results()

/home/simone/.local/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.VConcatChart(...)

0,z0
TA,0.8015
SW_IN,0.4251
LW_IN,-0.1879
VPD,1.0596
latent,lengthscale
z0,5.6057
variable,psi
TA,0.4174
SW_IN,0.8049
LW_IN,0.9361


In [ ]:
ai_r_gaps[0].plot_pred()

In [ ]:
hai_r_gaps[1].display_results()

In [ ]:
hai_r_gaps[2].display_results()

## Continous gap

In [ ]:
hai_c_gaps[0].display_results()

In [ ]:
hai_c_gaps[1].display_results()

In [ ]:
hai_c_gaps[2].display_results()